<a href="https://colab.research.google.com/github/saishshinde15/CrewAI/blob/main/Basic_customer_support_using_tools_and_agent_CrewAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.0/463.0 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [2]:
pip install langchain langchain-community

In [70]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()

from langchain_cohere import ChatCohere

model = ChatCohere()

··········


In [72]:
from crewai import Agent, Task, Crew

## Adding tools in task (the tools will be used when a particular task will be executed)

In [82]:
support_agent = Agent(
    role="Senior Support Representative",  # Assigning a 'Senior' role ensures higher quality outcomes.
    goal=(
        "Be the most friendly, helpful, and effective "
        "support representative on your team."
    ),
    backstory=(
        "You work at Tethys AI, a cutting-edge company dedicated to integrating AI into daily life. "
        "Currently, you're tasked with providing exceptional support to {customer}, "
        "one of the company's most valuable clients. "
        "Your mission is to deliver comprehensive, accurate, and empathetic assistance. "
        "Always provide complete answers and avoid making assumptions."
    ),
    llm=model,
    verbose=True,
    allow_delegation=False,  # Set to False to work independently without collaborating with other agents.
)

In [83]:
support_quality_assurance_agent = Agent(
    # Always include a QA agent in your framework to monitor and enhance support quality.
    role="Support Quality Assurance Specialist",
    goal=(
        "Be recognized as the most diligent and effective "
        "quality assurance specialist in your team."
    ),
    backstory=(
        "You work at Tethys AI, a leading company focused on integrating AI into daily life. "
        "Currently, you're collaborating with your team on a request from {customer}, "
        "ensuring that the support representative delivers the highest standard of service. "
        "Your role is to monitor interactions, verify that full, complete answers are provided, "
        "and ensure no assumptions are made. "
        "You are committed to maintaining and improving support quality."
    ),
    llm=model,
    verbose=True,
    allow_delegation=True,  # Set to False to work independently without agent collaboration.
)


In [84]:
from crewai_tools import ScrapeWebsiteTool

In [85]:
scraping_tool=ScrapeWebsiteTool(website_url='https://huggingface.co/blog/mlabonne/sft-llama3')

In [77]:
text = scraping_tool.run()
print(text)

Using Tool: Read website content
Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth
Hugging Face
					Models
					Datasets
					Spaces
					Posts
					Docs
					Enterprise
Pricing
Log In
Sign Up
						Back to Articles
		Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth
Community Article
Published
				July 29, 2024
		Upvote
		248
+242
mlabonne
Maxime Labonne
üîß Supervised Fine-Tuning
‚öñÔ∏è SFT Techniques
ü¶ô Fine-Tune Llama 3.1 8B
Conclusion
A beginner's guide to state-of-the-art supervised fine-tuning
The recent release of Llama 3.1 offers models with an incredible level of performance, closing the gap between closed-source and open-weight models. Instead of using frozen, general-purpose LLMs like GPT-4o and Claude 3.5, you can fine-tune Llama 3.1 for your specific use cases to achieve better performance and customizability at a lower cost.
In this article, we will provide a comprehensive overview of supervised fine-tuning. We will compare it to prompt engineering to understand wh

In [86]:
inquiry_resolution = Task(
    description=(
        "{customer} has reached out with a critical request:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one making this inquiry. "
        "Leverage all available resources and knowledge to deliver the best possible support. "
        "Your goal is to provide a thorough and accurate response, addressing every aspect of the inquiry."
    ),
    expected_output=(
        "A comprehensive, well-researched response that fully addresses the customer's inquiry. "
        "The response should reference all sources used, including external data or tools. "
        "Ensure the answer is detailed, leaving no part of the question unresolved. "
        "Maintain a friendly and professional tone throughout."
    ),
    tools=[scraping_tool],
    agent=support_agent
)


In [87]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure the response is thorough, accurate, and meets the high-quality standards expected of Tethys AI's customer support.\n"
        "Confirm that all aspects of the inquiry have been addressed comprehensively, maintaining a friendly and helpful tone.\n"
        "Verify the inclusion of references and sources used to gather information, ensuring the response is well-supported and leaves no questions unanswered."
    ),
    expected_output=(
        "A polished, detailed response ready for delivery to the customer.\n"
        "The final response should fully address the customer's inquiry, incorporating any relevant feedback or improvements.\n"
        "Maintain a professional yet approachable tone that reflects Tethys AI's chill and cool company culture."
    ),
    agent=support_quality_assurance_agent
)


In [88]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  #memory=True,
)

In [89]:
inputs = {
    "customer": "MetaAi",
    "person": "Saish",
    "inquiry": "I need help with how to finetune the model?"
}
result = crew.kickoff(inputs=inputs)

# Agent: Senior Support Representative
## Task: MetaAi has reached out with a critical request:
I need help with how to finetune the model?

Saish from MetaAi is the one making this inquiry. Leverage all available resources and knowledge to deliver the best possible support. Your goal is to provide a thorough and accurate response, addressing every aspect of the inquiry.


# Agent: Senior Support Representative
## Using tool: Read website content
## Tool Input: 
"{}O"
## Tool Output: 
Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth
Hugging Face
					Models
					Datasets
					Spaces
					Posts
					Docs
					Enterprise
Pricing
Log In
Sign Up
						Back to Articles
		Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth
Community Article
Published
				July 29, 2024
		Upvote
		248
+242
mlabonne
Maxime Labonne
üîß Supervised Fine-Tuning
‚öñÔ∏è SFT Techniques
ü¶ô Fine-Tune Llama 3.1 8B
Conclusion
A beginner's guide to state-of-the-art supervised fine-tuning
The recent release of Llama 3.1

In [91]:
print(result)

## Fine-tune Llama 3.1 Ultra-Efficiently with Unsloth

### 🍎🐧 Supervised Fine-Tuning (SFT)
Supervised Fine-Tuning is an advanced method to tailor Large Language Models (LLMs) for specific tasks. It involves retraining the model using a smaller dataset of instructions and desired outputs, enabling the model to follow instructions precisely and generate accurate responses. SFT is a powerful tool to enhance performance, introduce new knowledge, or adapt the model to unique tasks. Additionally, preference alignment can be employed to further refine the model by removing undesirable responses and adjusting the output style.

**Instruction Sample:**

| System Prompt | User Prompt | Expected Output |
|---|---|---|
| You are a helpful assistant that uses the provided information to answer questions. | What is the capital of France? | The capital of France is Paris. |

Prior to embarking on SFT, it is advisable to explore prompt engineering techniques such as few-shot prompting or RAG. These me